# Variational classifier

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

# Fitting the parity function

In [3]:
n_qubits = 4

In [4]:
dev = qml.device("default.qubit", wires=n_qubits)

### Functions to prepare the circuit

In [25]:
def layer(W):
    """Applies a layer of arbitrary rotations and circular entanglements to the variational circuit

    Args:
        W (np.ndarray): rotation parameters for the layer
    """
    for i in range(n_qubits):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)

    for i in range(n_qubits-1):
        qml.CNOT(wires=[i, i+1])

    if n_qubits > 2:
        qml.CNOT(wires=[n_qubits-1, 0])

In [6]:
def statepreparation(x):
    """Prepares the binary state fed to the vqc

    Args:
        x (List): list of 0s and 1s corresponding to the basis state
    """
    qml.BasisState(x, wires=[i for i in range(n_qubits)])

### Defining the `qnode`

**Exercise**: fill-in the following function with the state preparation routine and the layer routine. The weight array should be an array of 2D arrays. Each of the sub-array contains the parameters of a layer. Make sure to compute the expectation value of the $Z$ gate on qubit 0. Refer to [this link](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.expval.html).

In [19]:
@qml.qnode(dev)
def circuit(weights, x):
    """Generates the trainable quantum circuit with binary input

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        x (List): binaries representing the input
    """
    # ================
    # YOUR CODE BELOW
    # ================
    return #

### Adding a classical bias parameter

In [20]:
def variational_classifier(weights, bias, x):
    """Complete variational classifier, including a bias

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        bias (float): classical bias value, added to the output of the quantum circuit
        x (List): binaries representing the input
    """
    return circuit(weights, x) + bias

## Cost function

In [21]:
def square_loss(labels, predictions):
    """Computes the MSE between labels and predictions

    Args:
        labels (List[int]): actual values
        predictions (List[int]): model predictions

    Returns:
        float: value of the MSE
    """
    loss = 0.
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2

    loss /= len(labels)
    return loss

In [22]:
def accuracy(labels, predictions):
    """Returns the accuracy over the dataset

    Args:
        labels (List[int]): true values
        predictions (List[int]): model-predicted values

    Returns:
        float: number of accurate predictions over total
    """
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss += 1

    loss /= len(labels)
    return loss

**Exercise**: complete the cost function computing the MSE

In [23]:
def cost(weights, bias, X, Y):
    """Computes the predictions and returns the MSE over the dataset

    Args:
        weights (List[np.ndarray]): all the variational parameters, one
            array per layer
        bias (float): classical bias value, added to the output of the quantum circuit
        X (List[List]): list of all the binary input strings
        Y (List): labels

    Returns:
        float: MSE over the dataset
    """
    # ================
    # YOUR CODE BELOW
    # ================
    return #

## Optimization

In [7]:
data = np.loadtxt("variational_classifier/data/parity.txt")

In [8]:
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)

# shift lables from [0, 1] to [-1, 1], to match the range of expectation values
Y = 2 * Y - np.ones(len(Y))

### Initialization

In [ ]:
rng = np.random.default_rng(0)

num_layers = 2
weights_init = .01 * rng.normal(size=(num_layers, n_qubits, 3), requires_grad=True)
bias_init = np.array(0., requires_grad=True)

### Defining optimizer and batch size

In [17]:
opt = NesterovMomentumOptimizer(.5)
batch_size = 5

### Optimize

In [ ]:
weights = weights_init
bias = bias_init
iterations = 25

for it in range(iterations):

    # shuffle the batch indices
    batch_index = rng.integers(0, len(X), size=batch_size)

    X_batch = X[batch_index]
    Y_batch = Y[batch_index]

    # update weights and biases
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )